In [6]:
import pickle
from collections import Counter
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd
from IPython.core.display import display

from countries import get_countries

In [2]:
with open("data/actors.pkl", "rb") as f:
    df = pd.DataFrame(sorted(pickle.load(f)), columns=["nconst"])

def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
df = parallelize_dataframe(df, get_countries)

In [11]:
# df = pd.read_pickle("data/birth_countries.pkl")

df = df[df["birth_country"] != ""]
df["count"] = df["birth_country"].map(df["birth_country"].value_counts())
df = df[df["count"] >= 20]
df = df.reset_index().drop(["index", "count"], axis=1)

df.to_pickle("data/birth_countries.pkl")

In [16]:
# df = pd.read_pickle("data/birth_countries.pkl")

# pd.set_option("display.max_rows", None)
display(df)

countries = Counter(df["birth_country"].values).most_common()
for i, country in enumerate(countries):
    print(i, country)

,nconst,birth_country
0,nm0000001,USA
1,nm0000002,USA
2,nm0000003,France
3,nm0000006,Sweden
4,nm0000007,USA
...,...,...
10306,nm9144951,Japan
10307,nm9343226,Egypt
10308,nm9384165,Japan
10309,nm9535860,British India


0 ('USA', 2710)
1 ('Japan', 968)
2 ('UK', 635)
3 ('France', 585)
4 ('India', 582)
5 ('Italy', 497)
6 ('Germany', 394)
7 ('Turkey', 330)
8 ('Mexico', 279)
9 ('China', 272)
10 ('British India', 261)
11 ('Spain', 259)
12 ('Russia', 207)
13 ('Philippines', 206)
14 ('Canada', 157)
15 ('Egypt', 147)
16 ('Sweden', 126)
17 ('Argentina', 122)
18 ('Austria', 116)
19 ('Greece', 114)
20 ('Czech Republic', 113)
21 ('Hong Kong', 104)
22 ('Poland', 102)
23 ('Denmark', 100)
24 ('Taiwan', 89)
25 ('Australia', 75)
26 ('Hungary', 75)
27 ('Brazil', 70)
28 ('Iran', 61)
29 ('South Korea', 57)
30 ('Romania', 55)
31 ('Ukraine', 52)
32 ('Yugoslavia', 47)
33 ('Indonesia', 41)
34 ('Ireland', 39)
35 ('Belgium', 38)
36 ('Switzerland', 32)
37 ('Pakistan', 31)
38 ('Netherlands', 27)
39 ('Finland', 27)
40 ('Bulgaria', 23)
41 ('Austria-Hungary', 22)
42 ('British Crown Colony', 22)
43 ('Cuba', 21)
44 ('Bangladesh', 21)
